## Using ADQL to download Gaia data

This is the first in a series of lessons related to astronomy data.  As a running example, we will replicate part of the analysis in a recent paper, "[Off the beaten path: Gaia reveals GD-1 stars outside of the main stream](https://arxiv.org/abs/1805.00425)" by Adrian M. Price-Whelan and Ana Bonaca.

As the abstract explains, "Using data from the Gaia second data release combined with Pan-STARRS photometry, we present a sample of highly-probable members of the longest cold stream in the Milky Way, GD-1."

GD-1 is a [stellar stream](https://en.wikipedia.org/wiki/List_of_stellar_streams) which is "an association of stars orbiting a galaxy that was once a globular cluster or dwarf galaxy that has now been torn apart and stretched out along its orbit by tidal forces."

The two datasets used in this study are
 
* [Gaia](https://en.wikipedia.org/wiki/Gaia_(spacecraft)), which is "a space observatory of the European Space Agency (ESA), launched in 2013 ... designed for astrometry: measuring the positions, distances and motions of stars with unprecedented precision", and

* [PanSTARRS](https://en.wikipedia.org/wiki/Pan-STARRS): "The Panoramic Survey Telescope and Rapid Response System, located at Haleakala Observatory, Hawaii, US, consists of astronomical cameras, telescopes and a computing facility that is surveying the sky for moving or variable objects on a continual basis, and also producing accurate astrometry and photometry of already-detected objects."

Both of these datasets are very large, which can make them challenging to work with.  One of the goals of this workshop is to provide tools for working with large datasets.

One of the most important of those tools is a "query language", which is a way to query a large database and efficiently select the information you need.  So that's where we'll start.

The query language we'll use is ADQL, which stands for "Astronomical Data Query Language".

ADQL is a dialect of [SQL](https://en.wikipedia.org/wiki/SQL) (Structured Query Language), which is by far the most commonly used query language.  Almost everything you learn about ADQL also works in SQL.

[The reference manual for ADQL is here](https://docs.appdynamics.com/display/PRO45/ADQL+Reference).

But you might find it easier to learn from [this ADQL Cookbook](https://www.gaia.ac.uk/data/gaia-data-release-1/adql-cookbook).

## Getting Gaia data

The library we'll use to get Gaia data is [Astroquery](https://astroquery.readthedocs.io/en/latest/).  If you are running this notebook on your own computer, you might have to install Astroquery.  You should have received instructions for this before the workshop.

If you are running this notebook on Colab, you can run the following cell to install Astroquery and a couple of other libraries we'll use.

In [1]:
# If we're running on Colab, install libraries

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install astroquery astro-gala pyia
    !mkdir data

From Astroquery we can import `Gaia`, which is an [object that represents a connection to the Gaia database](https://astroquery.readthedocs.io/en/latest/gaia/gaia.html).

Running this import statement has the effect of creating a [TAP+](http://www.ivoa.net/documents/TAP/) connection; TAP stands for "Table Access Protocol".  It is a network protocol for sending queries to the database and getting back the results.

In [2]:
from astroquery.gaia import Gaia

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


I'm not sure why it seems to create two connections.

What is a database, anyway?  Most generally, it can be any collection of data, but when we are talking about ADQL or SQL:

* A database is a collection of one or more named tables.

* Each table is a 2-D array with one or more named columns of data.

We can use `load_tables` to get the names of the tables in the Gaia database.  With the option `only_names=True`, it loads information about the tables, the "metadata", not the data itself.

In [3]:
tables = Gaia.load_tables(only_names=True)
for table in (tables):
    print(table.get_qualified_name())

INFO: Retrieving tables... [astroquery.utils.tap.core]
INFO: Parsing tables... [astroquery.utils.tap.core]
INFO: Done. [astroquery.utils.tap.core]
external.external.apassdr9
external.external.gaiadr2_geometric_distance
external.external.galex_ais
external.external.ravedr5_com
external.external.ravedr5_dr5
external.external.ravedr5_gra
external.external.ravedr5_on
external.external.sdssdr13_photoprimary
external.external.skymapperdr1_master
external.external.tmass_xsc
public.public.hipparcos
public.public.hipparcos_newreduction
public.public.hubble_sc
public.public.igsl_source
public.public.igsl_source_catalog_ids
public.public.tycho2
public.public.dual
tap_config.tap_config.coord_sys
tap_config.tap_config.properties
tap_schema.tap_schema.columns
tap_schema.tap_schema.key_columns
tap_schema.tap_schema.keys
tap_schema.tap_schema.schemas
tap_schema.tap_schema.tables
gaiadr1.gaiadr1.aux_qso_icrf2_match
gaiadr1.gaiadr1.ext_phot_zero_point
gaiadr1.gaiadr1.allwise_best_neighbour
gaiadr1.gaiad

So that's a lot of tables.  The ones we'll use are:

* gaiadr2.gaia_source, which contains Gaia data from [data release 2](https://www.cosmos.esa.int/web/gaia/data-release-2),

* gaiadr2.panstarrs1_original_valid, which contains the photometry data we'll use from PanSTARRS, and

* gaiadr2.panstarrs1_best_neighbour, which we'll use to cross-match each star observed by Gaia with the same star observed by PanSTARRS.

We can use `load_table` (not `load_tables`) to get the metadata for a single table.  The name of this function is misleading, because it only downloads metadata. 

In [5]:
table = Gaia.load_table('gaiadr2.gaia_source')
table

Retrieving table 'gaiadr2.gaia_source'
Parsing table 'gaiadr2.gaia_source'...
Done.


Notice one gotcha: in the list of table names, this table appears as `gaiadr2.gaiadr2.gaia_source`, but when we load the metadata, we refer to it as `gaiadr2.gaia_source`.

Jupyter shows that the result is an object of type `TapTableMeta`, but it does not display the contents.

To see the metadata, we have to print the object.

In [6]:
print(table)

TAP Table name: gaiadr2.gaiadr2.gaia_source
Description: This table has an entry for every Gaia observed source as listed in the
Main Database accumulating catalogue version from which the catalogue
release has been generated. It contains the basic source parameters,
that is only final data (no epoch data) and no spectra (neither final
nor epoch).
Num. columns: 96


The following loop prints the names of the columns in the table.

In [9]:
for column in (table.columns):
    print(column.name)

solution_id
designation
source_id
random_index
ref_epoch
ra
ra_error
dec
dec_error
parallax
parallax_error
parallax_over_error
pmra
pmra_error
pmdec
pmdec_error
ra_dec_corr
ra_parallax_corr
ra_pmra_corr
ra_pmdec_corr
dec_parallax_corr
dec_pmra_corr
dec_pmdec_corr
parallax_pmra_corr
parallax_pmdec_corr
pmra_pmdec_corr
astrometric_n_obs_al
astrometric_n_obs_ac
astrometric_n_good_obs_al
astrometric_n_bad_obs_al
astrometric_gof_al
astrometric_chi2_al
astrometric_excess_noise
astrometric_excess_noise_sig
astrometric_params_solved
astrometric_primary_flag
astrometric_weight_al
astrometric_pseudo_colour
astrometric_pseudo_colour_error
mean_varpi_factor_al
astrometric_matched_observations
visibility_periods_used
astrometric_sigma5d_max
frame_rotator_object_type
matched_observations
duplicated_source
phot_g_n_obs
phot_g_mean_flux
phot_g_mean_flux_error
phot_g_mean_flux_over_error
phot_g_mean_mag
phot_bp_n_obs
phot_bp_mean_flux
phot_bp_mean_flux_error
phot_bp_mean_flux_over_error
phot_bp_mean_ma

To find out what the columns mean, you can read [the documentation of this table here](https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html).

## Writing queries

Now you might be wondering how we actually download the data.  With tables this big, you generally don't.  Instead, you use queries to select only the data you want.

To do that, we're going to write an ADQL query.  Here's an example:

In [11]:
query1 = """SELECT 
TOP 10
source_id, ref_epoch, ra, dec, parallax 
FROM gaiadr2.gaia_source"""

The words in uppercase are ADQL keywords:

* `SELECT` indicates that we are selecting data (as opposed to adding or modifying data).

* `TOP` indicates that we only want the first 10 rows of the table, which is useful for testing a query before asking for all of the data.

* `FROM` specifies which table we want data from.

The third line is a list of column names, indicating which columns we want.  

I wrote the column names in lowercase to make it clear that they are not keywords.  This is a common style, but it is not required.

To run this query, we use `Gaia.launch_job`:

In [12]:
job1 = Gaia.launch_job(query1)
job1

The result is an object that represents the job running on a Gaia server.

If you print it, it displays metadata for the forthcoming table.

In [13]:
print(job1)

<Table length=10>
           name            dtype  unit                            description                            
------------------------- ------- ---- ------------------------------------------------------------------
                source_id   int64      Unique source identifier (unique within a particular Data Release)
                ref_epoch float64   yr                                                    Reference epoch
                       ra float64  deg                                                    Right ascension
                      dec float64  deg                                                        Declination
                 parallax float64  mas                                                           Parallax
astrometric_n_good_obs_al   int32                                          Number of good observations AL
Jobid: None
Phase: COMPLETED
Owner: None
Output file: sync_20200727133344.xml.gz
Results: None


Now we can get the results:


In [15]:
results1 = job1.get_results()
type(results1)

astropy.table.table.Table

The results is an [Astropy Table](https://docs.astropy.org/en/stable/table/) is similar to a table in an SQL database except:

* SQL databases are stored on disk drives, so they are persistent; that is, they "survive" even if you turn off the computer.  An Astropy `Table` is stored in memory; it disappears when you turn off the computer (or shut down this Jupyter notebook).

* SQL databases are designed to process queries.  And Astropy `Table` can perform some query-like operations, like selecting columns and rows.  But these operations use Python syntax, not SQL.

Jupyter knows how to display the contents of a `Table`.

In [16]:
results1

source_id,ref_epoch,ra,dec,parallax,astrometric_n_good_obs_al
,yr,deg,deg,mas,
int64,float64,float64,float64,float64,int32
5778048606007762688,2015.5,252.93420276237282,-76.36404355513356,0.0073540961896056255,233
5778089219220100096,2015.5,252.72779858159538,-75.86754589201404,0.25581296749966514,253
5778038302382901760,2015.5,253.73048442254156,-75.97044898678163,0.19971165734613006,179
5778065137341689856,2015.5,252.6801881425296,-75.94965439295241,0.7703449958336754,241
5778053725609483136,2015.5,252.12098597753152,-76.28043425673263,0.27852627584949724,248
5778073521116914176,2015.5,251.2555423220548,-75.99925834891171,0.3851764561721036,283
5778088291507103360,2015.5,252.9473667300663,-75.89270654732523,-0.04894008708550348,266
5778084408856419584,2015.5,253.1933563035603,-75.99089099992894,0.4475173051231056,253


Each column has a name, units, and a data type.  

For example, the units of `ra` and `dec` are degrees, and their data type is `float64`, which is a 64-bit floating-point number, used to store measurements with a fraction part.

**Exercise:** Read [the documentation of this table](https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html) and select at least one column name that looks interesting to you.  Add the columns you selected to the query and run it again.  What are the units of the column you selected?  What is its data type?

## Asynchronous queries

`launch_job` asks the server to run the job "synchronously", which normally means it runs immediately.  But synchronous jobs are limited to 2000 rows.  For queries that return more rows, you have to run "asynchronously", which mean they might take longer to get started.

The results of an asynchronous query are stored in a file on the server, so you can start a query and come back later to get the results.

For anonymous users, files are kept for three days.

We could run the same query asynchronously, but to live things up, let's add a new keyword, `WHERE`:

In [11]:
query2 = """SELECT 
TOP 100
source_id, ref_epoch, ra, dec, parallax
WHERE parallax < 1
FROM gaiadr2.gaia_source"""

A `WHERE` clause indicates which rows we want; in this case, the query selects only rows "where" `parallax` is less than 1.

We use `launch_job_async` to submit an asynchronous query.

In [12]:
job2 = Gaia.launch_job_async(query2)
print(job2)

INFO: Query finished. [astroquery.utils.tap.core]
<Table length=10>
        name          dtype  unit                     description                    
-------------------- ------- ---- ---------------------------------------------------
         solution_id   int64                                      Solution Identifier
           ref_epoch float64   yr                                     Reference epoch
         ra_dec_corr float32      Correlation between right ascension and declination
astrometric_n_obs_al   int32                          Total number of observations AL
matched_observations   int16            Amount of observations matched to this source
Jobid: 1595337348686O
Phase: COMPLETED
Owner: None
Output file: async_20200721091548.vot
Results: None


In [13]:
results2 = job2.get_results()
results2

solution_id,ref_epoch,ra_dec_corr,astrometric_n_obs_al,matched_observations
,yr,,,
int64,float64,float32,int32,int16
1635721458409799680,2015.5,0.022670548,107,13
1635721458409799680,2015.5,0.06490505,97,11
1635721458409799680,2015.5,0.11690165,104,12
1635721458409799680,2015.5,0.042778816,114,13
1635721458409799680,2015.5,0.095711425,114,13
1635721458409799680,2015.5,0.56088775,105,12
1635721458409799680,2015.5,-0.0028029205,114,13
1635721458409799680,2015.5,0.11152559,104,12


Asynchronous jobs have a `jobid`.

In [14]:
job1.jobid, job2.jobid

(None, '1595337348686O')

Which you can use to remove the job from the server.

TODO: Confirm that this means the data file gets deleted.

In [15]:
Gaia.remove_jobs([job2.jobid])

Removed jobs: '['1595337348686O']'.


## Formatting queries

So far the queries have been string "literals", meaning that the entire string is part of the program.

It is more common to assemble a string programmatically using the string `format` method.

As an example, here's a list of columns we might want to select.

In [16]:
gaia_cols = ['source_id', 
             'ra',
             'dec',
             'pmra',
             'pmdec',
             'parallax', 
             'parallax_error', 
             ]

Before we assemble the string, we'll join the column names into a single string with commas and spaces.

In [17]:
columns = ', '.join(gaia_cols)
columns

'source_id, ra, dec, pmra, pmdec, parallax, parallax_error'

The following is a "base" for a query; it's a string that contains at least one format specification in curly brackets (braces).

In [18]:
query_base = """SELECT TOP 10
{columns}
FROM gaiadr2.gaia_source
WHERE parallax < 1 AND bp_rp > -0.75 AND bp_rp < 2
"""

To assemble the query, we invoke `format` on the base string and provide keyword arguments.

https://www.w3schools.com/python/ref_string_format.asp

Each keyword argument assigns a value to a variable.  Then we can access the variables in the format specification.

In [19]:
query3 = query_base.format(columns=columns)

We'll use the following function to print multi-line queries readably.

In [20]:
def print_query(query):
    for line in query.split('\n'):
        print(line)

Here's the query we just assembled.

In [21]:
print_query(query3)

SELECT TOP 10
source_id, ra, dec, pmra, pmdec, parallax, parallax_error
FROM gaiadr2.gaia_source
WHERE parallax < 1 AND bp_rp > -0.75 AND bp_rp < 2



The format specifier has been replaced with the value of `columns`.

We've already run this query, but we can run it again to confirm that our assembled query works.

In [22]:
job3 = Gaia.launch_job(query3)
print(job3)

<Table length=10>
     name       dtype    unit                              description                            
-------------- ------- -------- ------------------------------------------------------------------
     source_id   int64          Unique source identifier (unique within a particular Data Release)
            ra float64      deg                                                    Right ascension
           dec float64      deg                                                        Declination
          pmra float64 mas / yr                         Proper motion in right ascension direction
         pmdec float64 mas / yr                             Proper motion in declination direction
      parallax float64      mas                                                           Parallax
parallax_error float64      mas                                         Standard error of parallax
Jobid: None
Phase: COMPLETED
Owner: None
Output file: sync_20200721091551.xml.gz
Results: N

In [23]:
results3 = job3.get_results()
results3

source_id,ra,dec,pmra,pmdec,parallax,parallax_error
,deg,deg,mas / yr,mas / yr,mas,mas
int64,float64,float64,float64,float64,float64,float64
4042079415526077184,269.615505095578,-34.82083130836642,-0.14697065342756463,-1.15764447638223,0.5360805493350669,0.1426313351507156
4042078144213537024,269.72372980755273,-34.86014719868787,-5.701689461158032,-11.241933950766285,0.794452353833016,0.7151039689885
4042078831338090880,269.5872568621868,-34.86038675441301,-0.5007792418817473,-6.03553716652244,0.42235949110241516,0.09546467768883657
4042079110610736384,269.66145227862773,-34.83582264803566,-5.5855966785382485,-6.479323140794701,-0.31579370816381114,0.5651768495117003
4042078316011694592,269.6566823998595,-34.8644044274934,-4.071953658917412,-2.45009711436876,-0.4414312144711792,0.2436598460927262
2183266674116177152,310.2134344338742,53.796210648722976,-3.1579368338608047,-1.935493397184783,-0.1868137058665045,0.7463231800629537
2183285881199360256,310.1610282098487,54.07289702118478,-3.6447531685985943,-5.01828039877749,-0.11727829076448748,0.28502208396517553
2183274332033346560,310.46515505936054,53.93240964196606,-2.8141484339980076,-3.2144574697926886,0.06855306410247869,0.07017565320145407


## Working with coordinates

The next step is to select a viewing rectangle, but before we do that, we have to deal with coordinates.

In [24]:
import astropy.units as u

low1, high1 = -55, -45
phi1 = [low1, low1, high1, high1] * u.deg

low2, high2 = -5, 5
phi2 = [low2, high2, high2, low2] * u.deg

`gc.GD1Koposov10` is the [Astropy coordinate class for the Sagittarius coordinate system](https://gala-astro.readthedocs.io/en/latest/_modules/gala/coordinates/gd1.html)

In [25]:
import gala.coordinates as gc

corners = gc.GD1Koposov10(phi1=phi1, phi2=phi2)
type(corners)

gala.coordinates.gd1.GD1Koposov10

In [26]:
corners

<GD1Koposov10 Coordinate: (phi1, phi2) in deg
    [(-55., -5.), (-55.,  5.), (-45.,  5.), (-45., -5.)]>

Convert to [International Celestial Reference System](https://en.wikipedia.org/wiki/International_Celestial_Reference_System)

In [27]:
import astropy.coordinates as coord

corners_icrs = corners.transform_to(coord.ICRS)
type(corners_icrs)

astropy.coordinates.builtin_frames.icrs.ICRS

In [28]:
corners_icrs

<ICRS Coordinate: (ra, dec) in deg
    [(143.65740786, 20.98189113), (134.46717444, 26.39291777),
     (140.58825494, 34.85481377), (150.16628418, 29.01557079)]>

In [29]:
corners_icrs[0]

<ICRS Coordinate: (ra, dec) in deg
    (143.65740786, 20.98189113)>

In [30]:
corners_icrs[0].ra

<Longitude 143.65740786 deg>

In [31]:
corners_icrs[0].ra.degree

143.65740785846373

In [32]:
corners_icrs.ra

<Longitude [143.65740786, 134.46717444, 140.58825494, 150.16628418] deg>

In [33]:
corners_icrs.ra.degree

array([143.65740786, 134.46717444, 140.58825494, 150.16628418])

We can use `corners_icrs` to specify a polygon and construct a more complex query.

In [34]:
query4_base = """SELECT {columns}
FROM gaiadr2.gaia_source
WHERE parallax < 1 AND bp_rp > -0.75 AND bp_rp < 2 AND
      CONTAINS(POINT(ra, dec), 
               POLYGON({ra[0]}, {dec[0]}, 
                       {ra[1]}, {dec[1]}, 
                       {ra[2]}, {dec[2]}, 
                       {ra[3]}, {dec[3]})) = 1
"""

Here's what it looks like.

In [35]:
query4 = query4_base.format(columns=columns, 
                            ra=corners_icrs.ra.degree,
                            dec=corners_icrs.dec.degree)
print_query(query4)

SELECT source_id, ra, dec, pmra, pmdec, parallax, parallax_error
FROM gaiadr2.gaia_source
WHERE parallax < 1 AND bp_rp > -0.75 AND bp_rp < 2 AND
      CONTAINS(POINT(ra, dec), 
               POLYGON(143.65740785846373, 20.98189112798802, 
                       134.46717444171475, 26.39291776724364, 
                       140.58825494277238, 34.85481376928442, 
                       150.16628417989443, 29.015570791894923)) = 1



And here's how we run it.

In [36]:
job4 = Gaia.launch_job_async(query4)
print(job4)

INFO: Query finished. [astroquery.utils.tap.core]
<Table length=120756>
     name       dtype    unit                              description                            
-------------- ------- -------- ------------------------------------------------------------------
     source_id   int64          Unique source identifier (unique within a particular Data Release)
            ra float64      deg                                                    Right ascension
           dec float64      deg                                                        Declination
          pmra float64 mas / yr                         Proper motion in right ascension direction
         pmdec float64 mas / yr                             Proper motion in declination direction
      parallax float64      mas                                                           Parallax
parallax_error float64      mas                                         Standard error of parallax
Jobid: 1595337353369O
Phase: COMPLETE

In [37]:
results4 = job4.get_results()
len(results4)

120756

## Saving results

In [38]:
filename = 'data/gd1_results4.fits'
results4.write(filename, overwrite=True)

In [39]:
import os

def filesize(filename):
    size = os.path.getsize(filename)
    print(size / 1024 / 1024, 'MB')

In [40]:
filesize(filename)

6.46270751953125 MB


In [41]:
from astropy.table import Table

filename = 'data/gd1_results4.fits'
results4 = Table.read(filename)

In [42]:
results4.info

<Table length=120756>
     name       dtype    unit                              description                            
-------------- ------- -------- ------------------------------------------------------------------
     source_id   int64          Unique source identifier (unique within a particular Data Release)
            ra float64      deg                                                    Right ascension
           dec float64      deg                                                        Declination
          pmra float64 mas / yr                         Proper motion in right ascension direction
         pmdec float64 mas / yr                             Proper motion in declination direction
      parallax float64      mas                                                           Parallax
parallax_error float64      mas                                         Standard error of parallax

## Making a function

In [43]:
def transform_rectangle(low1, high1, low2, high2):
    phi1 = [low1, low1, high1, high1] * u.deg
    phi2 = [low2, high2, high2, low2] * u.deg
    corners = gc.GD1Koposov10(phi1=phi1, phi2=phi2)
    corners_icrs = corners.transform_to(coord.ICRS)
    return corners_icrs

In [44]:
corners_icrs = transform_rectangle(-55, -45, -4, 6)

In [45]:
assert(corners_icrs[0].ra.degree == 142.7716385024318)

In [46]:
point_base = "{point.ra.degree}, {point.dec.degree}"

t = [point_base.format(point=point)
     for point in corners_icrs]
print(t)

['142.7716385024318, 21.546353324354072', '133.5042010718342, 26.902060612630827', '139.5603146332043, 35.39626209390598', '149.26169474640096, 29.63037759884082']


In [47]:
poly_base = "POLYGON({point_list})"

point_list = ', '.join(t)
poly_base.format(point_list=point_list)

'POLYGON(142.7716385024318, 21.546353324354072, 133.5042010718342, 26.902060612630827, 139.5603146332043, 35.39626209390598, 149.26169474640096, 29.63037759884082)'

In [48]:
def make_adql_polygon(coords):
    """
    """
    point_base = "{point.ra.degree}, {point.dec.degree}"

    t = [point_base.format(point=point)
         for point in coords]

    poly_base = "POLYGON({point_list})"
    point_list = ', '.join(t)
    return poly_base.format(point_list=point_list)

In [49]:
polygon1 = make_adql_polygon(corners_icrs)

In [50]:
corners_icrs = transform_rectangle(-55, -45, -5, 5)
polygon2 = make_adql_polygon(corners_icrs)
polygon2

'POLYGON(143.65740785846373, 20.98189112798802, 134.46717444171475, 26.39291776724364, 140.58825494277238, 34.85481376928442, 150.16628417989443, 29.015570791894923)'